<h2>This notebook used to analyse the official NBA League statistics.<h2>

In [32]:
import pandas as pd
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.static import teams
import datetime

<h4>Lets make a df contains the games<h4>

In [33]:
log = leaguegamelog.LeagueGameLog().get_normalized_dict()['LeagueGameLog']

df1 = pd.DataFrame(log[::2])
df2 =  pd.DataFrame(log[1::2])
df2.columns = [f"{column}_2" for column in df2.columns]

all_games = df1.join(df2)
all_games['WL'] = all_games['WL'].map(lambda x: 'H' if x == 'W' else 'A')
all_games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB_2,REB_2,AST_2,STL_2,BLK_2,TOV_2,PF_2,PTS_2,PLUS_MINUS_2,VIDEO_AVAILABLE_2
0,22021,1610612744,GSW,Golden State Warriors,0022100002,2021-10-19,GSW @ LAL,H,240,41,...,40,45,21,7,4,18,25,114,-7,1
1,22021,1610612749,MIL,Milwaukee Bucks,0022100001,2021-10-19,MIL vs. BKN,H,240,48,...,39,44,19,3,9,13,17,104,-23,1
2,22021,1610612754,IND,Indiana Pacers,0022100003,2021-10-20,IND @ CHA,A,240,42,...,34,46,29,9,5,8,21,123,1,1
3,22021,1610612741,CHI,Chicago Bulls,0022100004,2021-10-20,CHI @ DET,H,240,37,...,36,47,17,7,5,17,16,88,-6,1
4,22021,1610612761,TOR,Toronto Raptors,0022100006,2021-10-20,TOR vs. WAS,A,240,30,...,44,51,20,14,8,22,17,98,15,1


<h4>Pick some useful data to train a model<h4>

In [44]:
all_games_useful = all_games[['TEAM_ID','TEAM_ID_2','GAME_DATE','WL']]
all_games_useful['GAME_DATE'] = pd.to_datetime(all_games_useful['GAME_DATE'], format="%Y-%m-%d")
all_games_useful['day_in_month'] = all_games_useful['GAME_DATE'].map(lambda x: x.strftime("%d"))
all_games_useful['day_in_week'] = all_games_useful['GAME_DATE'].map(lambda x: x.strftime("%w"))
all_games_useful.head(10)

C:\Users\milan\AppData\Local\Temp/ipykernel_6944/863627650.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_games_useful['GAME_DATE'] = pd.to_datetime(all_games_useful['GAME_DATE'], format="%Y-%m-%d")
C:\Users\milan\AppData\Local\Temp/ipykernel_6944/863627650.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_games_useful['day_in_month'] = all_games_useful['GAME_DATE'].map(lambda x: x.strftime("%d"))
C:\Users\milan\AppData\Local\Temp/ipykernel_6944/863627650.py:4: SettingWithCopyWarning: 
A valu

,TEAM_ID,TEAM_ID_2,GAME_DATE,WL,day_in_month,day_in_week
0,1610612744,1610612747,2021-10-19,H,19,2
1,1610612749,1610612751,2021-10-19,H,19,2
2,1610612754,1610612766,2021-10-20,A,20,3
3,1610612741,1610612765,2021-10-20,H,20,3
4,1610612761,1610612764,2021-10-20,A,20,3
5,1610612740,1610612755,2021-10-20,A,20,3
6,1610612763,1610612739,2021-10-20,H,20,3
7,1610612750,1610612745,2021-10-20,H,20,3
8,1610612752,1610612738,2021-10-20,H,20,3
9,1610612753,1610612759,2021-10-20,A,20,3


<h4>Split to train and test<h4>

In [45]:
X = all_games_useful[['TEAM_ID','TEAM_ID_2','day_in_month', 'day_in_week']]
y = all_games_useful[['WL']]

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

<h4>Build a model and fit with the values<h4>

In [26]:
from sklearn.neighbors import KNeighborsClassifier

In [27]:
model_knn = KNeighborsClassifier(n_neighbors=3).fit(X_train,y_train)

C:\Users\milan\.virtualenvs\prog_fileok-u0ABPTBr\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


<h4>Get the team ids and the date(day)<h4>

In [29]:
team_1 = teams.find_teams_by_full_name("Denver Nugets")
team_2 = teams.find_teams_by_full_name("GoldenState Warriors")
date_day = datetime.now()

In [ ]:
model_knn.predict([])

In [1]:
import nba_analysis_helpers as test

In [2]:
s = test.get_team_log("Denver Nuggets")